In [1]:
"""
AUTHOR: Pranav Parthasarathy
        Vedaad Shakib
        Shubhrakanti Ganguly
        Alexander Bondarenko
        Maksim Ivanov
FUNCTION: Combine and standardize all features into one notebook, perform PCA on these features
"""
import pandas as pd
import numpy as np
import urllib as u
from bs4 import BeautifulSoup as bs
import matplotlib.pyplot as plt
import lxml

from IPython.display import display

In [2]:
#Number of equities we want to suggest. AKA. How many we choose to output with our ranking metric

NUM_RANKED = 10

In [3]:
#get all the CSVs as dataframes 
constituents = pd.read_csv("constituents.csv")
fundamentals = pd.read_csv("fundamentals.csv")
prices_split_adjusted = pd.read_csv("prices-split-adjusted.csv")
securities = pd.read_csv("securities.csv")

In [4]:
# sample portfolio
portfolio = {'TDG': 0.50, 'ALK': 0.25, 'K': .25}

In [7]:
#merge all the data sets into one large one by ticker symbol and only keep the union 
fundamentals = fundamentals[['Ticker Symbol']]
universe = set(securities['Ticker symbol']) \
         & set(prices_split_adjusted['symbol']) \
         & set(fundamentals['Ticker Symbol']) \
         & set(constituents['Symbol'])

In [8]:
def historical_volatility(df):
    p = np.array(df.close)
    lr = np.log(p[1:]) - np.log(p[:-1])
    return np.sum(np.square(lr))
    
def covariance(df1, df2):
    date1 = np.array(df1.date)
    date2 = np.array(df2.date)
    start = max(date1[0], date2[0])
    end = min(date1[-1], date2[-1])
    p1 = np.array(df1[(start <= df1.date) & (df1.date <= end)].close)
    p2 = np.array(df2[(start <= df2.date) & (df2.date <= end)].close)
    lr1 = np.log(p1[1:]) - np.log(p1[:-1])
    lr2 = np.log(p2[1:]) - np.log(p2[:-1])
    return np.sum(lr1*lr2)

In [333]:
vol = []
portfolio_prices = {s: prices_split_adjusted[prices_split_adjusted.symbol == s].sort_values(['date'], ascending=[True]) for s in portfolio.keys()}
for s in universe:
    df = prices_split_adjusted[prices_split_adjusted.symbol == s]
    df.sort_values(['date'], ascending=[True], inplace=True)
    vol.append({
        'symbol': s,
        'volatility': historical_volatility(df),
        'added_volatility': sum(w * covariance(df, portfolio_prices[s]) for s, w in portfolio.items()),
    })
vol = pd.DataFrame(vol, columns = ['symbol', 'volatility', 'added_volatility'])
vol

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


,symbol,volatility,added_volatility
0,TDG,0.228652,0.142494
1,ALK,0.758006,0.250980
2,LRCX,0.666669,0.137728
3,CPB,0.200839,0.069950
4,K,0.181806,0.086913
5,UTX,0.274869,0.101148
6,XOM,0.253286,0.074361
7,AAPL,0.482109,0.092400
8,CMA,0.612700,0.130315
9,NTAP,0.731537,0.108066


In [334]:
securities.columns

Index(['Ticker symbol', 'Security', 'SEC filings', 'GICS Sector',
       'GICS Sub Industry', 'Address of Headquarters', 'Date first added',
       'CIK'],
      dtype='object')

In [335]:
securities = securities[['Ticker symbol', 'GICS Sector', 'GICS Sub Industry']]

In [336]:
def merge(keep_key, dfs_by_key):
    merged = dfs_by_key[0][1].copy()
    for k, df in dfs_by_key[1:]:
        merged = merged.merge(df, left_on=dfs_by_key[0][0], right_on=k, how='inner')
        if k != keep_key:
            del merged[k]
    if dfs_by_key[0][0] != keep_key:
        del merged[dfs_by_key[0][0]]
    return merged

In [337]:
df = merge('symbol', [('symbol', vol), ('Symbol', constituents), ('Ticker symbol', securities)])
df.head()

,symbol,volatility,added_volatility,Name,Sector,GICS Sector,GICS Sub Industry
0,TDG,0.228652,0.142494,TransDigm Group,Industrials,Industrials,Aerospace & Defense
1,ALK,0.758006,0.250980,Alaska Air Group Inc,Industrials,Industrials,Airlines
2,LRCX,0.666669,0.137728,Lam Research,Information Technology,Information Technology,Semiconductor Equipment
3,CPB,0.200839,0.069950,Campbell Soup,Consumer Staples,Consumer Staples,Packaged Foods & Meats
4,K,0.181806,0.086913,Kellogg Co.,Consumer Staples,Consumer Staples,Packaged Foods & Meats


In [338]:
volatility_df = df

In [339]:
df_sec = pd.read_csv("securities.csv")
df_fund = pd.read_csv("fundamentals.csv")
df_price = pd.read_csv("prices.csv")

df_price = df_price[['symbol','close']].drop_duplicates(subset='symbol', keep = "last")
df = pd.DataFrame()
df["Ticker"] = df_fund["Ticker Symbol"]
df["EPS"] = df_fund['Earnings Per Share']
df = df.drop_duplicates(subset='Ticker', keep = "last")
df["Industry"] = np.nan
df["Price"] = np.nan

In [340]:
for x in df_sec.iterrows():
    truth = df["Ticker"]==x[1]['Ticker symbol']
    try:
        df.loc[df.loc[truth]["Ticker"].index.values[0],'Industry'] = x[1]['GICS Sector']
    except:
        pass

for x in df_price.iterrows():
    truth = df["Ticker"]==x[1]['symbol']
    try:
        df.loc[df.loc[truth]["Ticker"].index.values[0],'Price'] = x[1]['close']
    except:
        pass


df = df.drop(df[df.EPS < 0].index)
df["PE"] = round(df['Price']/df['EPS'], 2)
df = df.dropna()

# Calculate industry averages
industries_dict = {sector: np.mean(df[df.Industry == sector].PE.values) for sector in df.Industry.unique()}
df.head()

,Ticker,EPS,Industry,Price,PE
3,AAL,11.39,Industrials,46.689999,4.10
7,AAP,6.45,Consumer Discretionary,169.119995,26.22
11,AAPL,8.35,Information Technology,115.820000,13.87
15,ABBV,3.15,Health Care,62.619999,19.88
19,ABC,6.73,Health Care,78.190002,11.62


In [341]:
volatility_df.head()

,symbol,volatility,added_volatility,Name,Sector,GICS Sector,GICS Sub Industry
0,TDG,0.228652,0.142494,TransDigm Group,Industrials,Industrials,Aerospace & Defense
1,ALK,0.758006,0.250980,Alaska Air Group Inc,Industrials,Industrials,Airlines
2,LRCX,0.666669,0.137728,Lam Research,Information Technology,Information Technology,Semiconductor Equipment
3,CPB,0.200839,0.069950,Campbell Soup,Consumer Staples,Consumer Staples,Packaged Foods & Meats
4,K,0.181806,0.086913,Kellogg Co.,Consumer Staples,Consumer Staples,Packaged Foods & Meats


In [342]:
volatility_pe_df = volatility_df.merge(df, how='inner', left_on=['symbol'], right_on=['Ticker'])

In [343]:
volatility_pe_df.head()

,symbol,volatility,added_volatility,Name,Sector,GICS Sector,GICS Sub Industry,Ticker,EPS,Industry,Price,PE
0,TDG,0.228652,0.142494,TransDigm Group,Industrials,Industrials,Aerospace & Defense,TDG,2.77,Industrials,248.960007,89.88
1,ALK,0.758006,0.250980,Alaska Air Group Inc,Industrials,Industrials,Airlines,ALK,6.61,Industrials,88.730003,13.42
2,LRCX,0.666669,0.137728,Lam Research,Information Technology,Information Technology,Semiconductor Equipment,LRCX,5.75,Information Technology,105.730003,18.39
3,CPB,0.200839,0.069950,Campbell Soup,Consumer Staples,Consumer Staples,Packaged Foods & Meats,CPB,1.82,Consumer Staples,60.470001,33.23
4,K,0.181806,0.086913,Kellogg Co.,Consumer Staples,Consumer Staples,Packaged Foods & Meats,K,1.74,Consumer Staples,73.709999,42.36


In [344]:
def get_standarized_PE(ticker, pe= None, industry= None):
    if not pe or not industry:
        assert ticker in df.Ticker.values, "Please provide price and Industry"
        data = df.loc[df["Ticker"] == ticker]
        indus = data["Industry"].values[0]
        PE = data["PE"].values[0]
        avg = industries_dict[indus]
        return (PE - avg)/avg
    else:
        assert industry in industries_sect.keyes(), "You're sector needs to be a GICS Sector"
        avg = industries_dict[industry]
        return (pe - avg)/avg

In [345]:
col = []
for ticker in volatility_pe_df['symbol']:
    col.append(get_standarized_PE(ticker))
volatility_pe_df['std_pe'] = np.array(col)

In [346]:
volatility_pe_df.head()

,symbol,volatility,added_volatility,Name,Sector,GICS Sector,GICS Sub Industry,Ticker,EPS,Industry,Price,PE,std_pe
0,TDG,0.228652,0.142494,TransDigm Group,Industrials,Industrials,Aerospace & Defense,TDG,2.77,Industrials,248.960007,89.88,2.446119
1,ALK,0.758006,0.250980,Alaska Air Group Inc,Industrials,Industrials,Airlines,ALK,6.61,Industrials,88.730003,13.42,-0.485459
2,LRCX,0.666669,0.137728,Lam Research,Information Technology,Information Technology,Semiconductor Equipment,LRCX,5.75,Information Technology,105.730003,18.39,-0.373890
3,CPB,0.200839,0.069950,Campbell Soup,Consumer Staples,Consumer Staples,Packaged Foods & Meats,CPB,1.82,Consumer Staples,60.470001,33.23,0.231245
4,K,0.181806,0.086913,Kellogg Co.,Consumer Staples,Consumer Staples,Packaged Foods & Meats,K,1.74,Consumer Staples,73.709999,42.36,0.569531


In [347]:
fifty_two_week_df = pd.read_csv("prices-split-adjusted.csv")
fifty_two_week_df['date'] = pd.to_datetime(fifty_two_week_df.date)

#Keep only the data from 2016 (we only need 1 years worth of data)
mask = (fifty_two_week_df['date'] > '2015-12-31')
fifty_two_week_df = fifty_two_week_df.loc[mask]


#Keep only the high and low and merge it all into one final data frame
s1 = fifty_two_week_df.groupby(['symbol'], sort=False)['close'].max()
s2 = fifty_two_week_df.groupby(['symbol'], sort=False)['close'].min()
final = pd.concat([s1, s2], axis=1).reset_index()
final.columns = ['symbol','high','low']

fifty_two_week_volatility_pe_df = final.merge(
    volatility_pe_df, how='inner', left_on=['symbol'], right_on=['symbol'])
fifty_two_week_volatility_pe_df.head()

,symbol,high,low,volatility,added_volatility,Name,Sector,GICS Sector,GICS Sub Industry,Ticker,EPS,Industry,Price,PE,std_pe
0,AAL,49.639999,25.270000,1.642028,0.250758,American Airlines Group,Industrials,Industrials,Airlines,AAL,11.39,Industrials,46.689999,4.10,-0.842801
1,AAP,176.779999,136.190002,0.519604,0.093686,Advance Auto Parts,Consumer Discretionary,Consumer Discretionary,Automotive Retail,AAP,6.45,Consumer Discretionary,169.119995,26.22,0.170151
2,AAPL,118.250000,90.339996,0.482109,0.092400,Apple Inc.,Information Technology,Information Technology,Computer Hardware,AAPL,8.35,Information Technology,115.820000,13.87,-0.527779
3,ABBV,67.389999,51.180000,0.302683,0.064782,AbbVie,Health Care,Health Care,Pharmaceuticals,ABBV,3.15,Health Care,62.619999,19.88,-0.335260
4,ABC,103.360001,69.029999,0.310987,0.072112,AmerisourceBergen Corp,Health Care,Health Care,Health Care Distributors,ABC,6.73,Health Care,78.190002,11.62,-0.611455


In [348]:
sector_feature = []
sector_weights = {}
for ticker in portfolio:
    sector = list(fifty_two_week_volatility_pe_df[fifty_two_week_volatility_pe_df.symbol == ticker]['Sector'])[0]
    if sector in sector_weights:
        sector_weights[sector] += portfolio[ticker]
    else:
        sector_weights[sector] = portfolio[ticker]
for x in fifty_two_week_volatility_pe_df.iterrows():
    if x[1]['Sector'] in sector_weights:
        sector_feature.append(sector_weights[x[1]['Sector']])
    else:
        sector_feature.append(0)
fifty_two_week_volatility_pe_df['portfolio_sector'] = sector_feature

In [349]:
fifty_two_week_volatility_pe_df.head()

,symbol,high,low,volatility,added_volatility,Name,Sector,GICS Sector,GICS Sub Industry,Ticker,EPS,Industry,Price,PE,std_pe,portfolio_sector
0,AAL,49.639999,25.270000,1.642028,0.250758,American Airlines Group,Industrials,Industrials,Airlines,AAL,11.39,Industrials,46.689999,4.10,-0.842801,0.75
1,AAP,176.779999,136.190002,0.519604,0.093686,Advance Auto Parts,Consumer Discretionary,Consumer Discretionary,Automotive Retail,AAP,6.45,Consumer Discretionary,169.119995,26.22,0.170151,0.00
2,AAPL,118.250000,90.339996,0.482109,0.092400,Apple Inc.,Information Technology,Information Technology,Computer Hardware,AAPL,8.35,Information Technology,115.820000,13.87,-0.527779,0.00
3,ABBV,67.389999,51.180000,0.302683,0.064782,AbbVie,Health Care,Health Care,Pharmaceuticals,ABBV,3.15,Health Care,62.619999,19.88,-0.335260,0.00
4,ABC,103.360001,69.029999,0.310987,0.072112,AmerisourceBergen Corp,Health Care,Health Care,Health Care Distributors,ABC,6.73,Health Care,78.190002,11.62,-0.611455,0.00


In [350]:
"""
standardize unstandardized features
"""
vol_mean, vol_std = fifty_two_week_volatility_pe_df[
    'volatility '].mean(), fifty_two_week_volatility_pe_df['volatility'].std()
high_low_diff = [x/y for x,y in zip(fifty_two_week_volatility_pe_df['high'], fifty_two_week_volatility_pe_df['low'])]
high_low_mean, high_low_std = pd.Series(high_low_diff).mean(), pd.Series(high_low_diff).std()
sector_mean, sector_std = fifty_two_week_volatility_pe_df[
    'portfolio_sector'].mean(), fifty_two_week_volatility_pe_df['portfolio_sector'].std()

std_high_low, std_vol, std_sector = [],[],[]
for x in fifty_two_week_volatility_pe_df.iterrows():
    std_high_low.append((x[1]['high']/x[1]['low']-high_low_mean)/high_low_std)
    std_vol.append((x[1]['volatility']-vol_mean)/vol_std)
    std_sector.append((x[1]['portfolio_sector']-sector_mean)/sector_std)
    
fifty_two_week_volatility_pe_df['std_high_low'] = np.array(std_high_low)
fifty_two_week_volatility_pe_df['std_vol'] = np.array(std_vol)
fifty_two_week_volatility_pe_df['std_sector'] = np.array(std_sector)

In [360]:
"""
Now make features (52 week high/low + volatility + portfolio_sector + std_pe) and 
perform Latent Factor Analysis
"""

X = []
centroid = {}
for x in fifty_two_week_volatility_pe_df.iterrows():
    point = np.array([
        x[1]['std_high_low'], x[1]['std_vol'], x[1]['std_pe'], x[1]['std_sector']])
    if x[1]['symbol'] in portfolio:
        centroid[tuple(point)] = portfolio[x[1]['symbol']]
    X.append(point)
    
center = [0,0,0,0]
for pt in centroid:
    for i in range(4):
        center[i] += centroid[pt]*pt[i]
        
X = np.array(X)
U, s, V = np.linalg.svd(X, full_matrices=True)
shifted_center = []
for i in range(4):
    shifted_center.append(V[-1][i]*s[-1]+center[i])
    
dist = {}
for i in range(len(X)):
    c_dist = 0
    for j in range(4):
        c_dist += (X[i][j]-shifted_center[j])**2
    dist[i] = c_dist
dist = sorted(dist, key = lambda x: dist[x])
rankings = []
for i in range(NUM_RANKED):
    rankings.append(fifty_two_week_volatility_pe_df.iloc[dist[i]])
pd.DataFrame(rankings)

,symbol,high,low,volatility,added_volatility,Name,Sector,GICS Sector,GICS Sub Industry,Ticker,EPS,Industry,Price,PE,std_pe,portfolio_sector,std_high_low,std_vol,std_sector
181,NAVI,17.850000,8.410000,0.274734,0.044267,Navient,Financials,Financials,Consumer Finance,NAVI,2.66,Financials,16.430000,6.18,-0.668901,0.00,1.993024,-0.784639,-0.406533
122,HPE,24.680000,12.060000,0.163954,0.032783,Hewlett Packard Enterprise,Information Technology,Information Technology,"Technology Hardware, Storage & Peripherals",HPE,1.84,Information Technology,23.139999,12.58,-0.571698,0.00,1.752610,-1.177965,-0.406533
284,XYL,54.750000,32.799999,0.205093,0.066929,Xylem Inc.,Industrials,Industrials,Industrial Conglomerates,XYL,1.88,Industrials,49.520000,26.34,0.009911,0.75,0.559917,-1.031900,2.827084
231,SRCL,128.199997,71.610001,0.396401,0.093723,Stericycle Inc,Industrials,Industrials,Industrial Conglomerates,SRCL,3.02,Industrials,77.040001,25.51,-0.021913,0.75,0.942631,-0.352658,2.827084
62,CTAS,120.709999,80.330002,0.302018,0.101844,Cintas Corporation,Industrials,Industrials,Diversified Support Services,CTAS,6.30,Industrials,115.559998,18.34,-0.296820,0.75,0.033390,-0.687767,2.827084
177,MPC,51.240002,30.730000,0.489823,0.076475,Marathon Petroleum,Energy,Energy,Oil & Gas Refining & Marketing & Transportation,MPC,5.29,Energy,50.349998,9.52,-0.842436,0.00,0.554285,-0.020964,-0.406533
46,CFG,36.459999,18.139999,0.160348,0.034765,Citizens Financial Group,Financials,Financials,Regional Banks,CFG,1.55,Financials,35.630001,22.99,0.231709,0.00,1.637170,-1.190769,-0.406533
16,AMAT,33.330002,15.640000,0.555650,0.129890,Applied Materials Inc,Information Technology,Information Technology,Semiconductor Equipment,AMAT,1.56,Information Technology,32.270000,20.69,-0.295583,0.00,2.020219,0.212757,-0.406533
236,SYF,37.259998,23.360001,0.148854,0.036540,Synchrony Financial,Financials,Financials,Consumer Finance,SYF,2.66,Financials,36.270000,13.64,-0.269225,0.00,0.325401,-1.231578,-0.406533
7,ADM,47.720001,30.510000,0.436517,0.104981,Archer-Daniels-Midland Co,Consumer Staples,Consumer Staples,Agricultural Products,ADM,2.99,Consumer Staples,45.650002,15.27,-0.434213,0.25,0.227524,-0.210227,0.671339


In [368]:
fifty_two_week_volatility_pe_df.head()

,symbol,high,low,volatility,added_volatility,Name,Sector,GICS Sector,GICS Sub Industry,Ticker,EPS,Industry,Price,PE,std_pe,portfolio_sector,std_high_low,std_vol,std_sector
0,AAL,49.639999,25.270000,1.642028,0.250758,American Airlines Group,Industrials,Industrials,Airlines,AAL,11.39,Industrials,46.689999,4.10,-0.842801,0.75,1.493190,4.069959,2.827084
1,AAP,176.779999,136.190002,0.519604,0.093686,Advance Auto Parts,Consumer Discretionary,Consumer Discretionary,Automotive Retail,AAP,6.45,Consumer Discretionary,169.119995,26.22,0.170151,0.00,-0.613618,0.084775,-0.406533
2,AAPL,118.250000,90.339996,0.482109,0.092400,Apple Inc.,Information Technology,Information Technology,Computer Hardware,AAPL,8.35,Information Technology,115.820000,13.87,-0.527779,0.00,-0.579140,-0.048352,-0.406533
3,ABBV,67.389999,51.180000,0.302683,0.064782,AbbVie,Health Care,Health Care,Pharmaceuticals,ABBV,3.15,Health Care,62.619999,19.88,-0.335260,0.00,-0.554538,-0.685406,-0.406533
4,ABC,103.360001,69.029999,0.310987,0.072112,AmerisourceBergen Corp,Health Care,Health Care,Health Care Distributors,ABC,6.73,Health Care,78.190002,11.62,-0.611455,0.00,0.016455,-0.655923,-0.406533


In [353]:
V

array([[-0.70801   , -0.70067874, -0.07026806,  0.05323101],
       [ 0.08747784, -0.01720813,  0.04396789,  0.9950469 ],
       [-0.5596556 ,  0.625716  , -0.53689083,  0.08374562],
       [ 0.42172868, -0.34239256, -0.83956981, -0.00589896]])

In [354]:
fifty_two_week_volatility_pe_df.loc[fifty_two_week_volatility_pe_df['Sector'] == 'Energy']


,symbol,high,low,volatility,added_volatility,Name,Sector,GICS Sector,GICS Sub Industry,Ticker,EPS,Industry,Price,PE,std_pe,portfolio_sector,std_high_low,std_vol,std_sector
65,CVX,118.769997,78.980003,0.332929,0.079788,Chevron Corp.,Energy,Energy,Integrated Oil & Gas,CVX,2.46,Energy,117.699997,47.85,-0.208044,0.0,0.036938,-0.578016,-0.406533
66,CXO,143.250000,72.519997,1.034613,0.118226,Concho Resources,Energy,Energy,Oil & Gas Exploration & Production,CXO,0.54,Energy,132.600006,245.56,3.064217,0.0,1.527756,1.913322,-0.406533
177,MPC,51.240002,30.730000,0.489823,0.076475,Marathon Petroleum,Energy,Energy,Oil & Gas Refining & Marketing & Transportation,MPC,5.29,Energy,50.349998,9.52,-0.842436,0.0,0.554285,-0.020964,-0.406533
193,OKE,59.029999,19.620001,0.745463,0.120721,ONEOK,Energy,Energy,Oil & Gas Exploration & Production,OKE,1.17,Energy,57.410000,49.07,-0.187852,0.0,4.794927,0.886691,-0.406533
209,PSX,90.160004,72.900002,0.284013,0.066928,Phillips 66,Energy,Energy,Oil & Gas Refining & Marketing & Transportation,PSX,7.78,Energy,86.410004,11.11,-0.816120,0.0,-0.807359,-0.751693,-0.406533
250,TSO,107.099998,68.639999,1.090175,0.133651,Tesoro Petroleum Co.,Energy,Energy,Oil & Gas Refining & Marketing & Transportation,TSO,12.50,Energy,87.449997,7.00,-0.884144,0.0,0.215627,2.110596,-0.406533
263,VLO,72.089996,47.240002,0.886007,0.133513,Valero Energy,Energy,Energy,Oil & Gas Refining & Marketing & Transportation,VLO,8.00,Energy,68.320000,8.54,-0.858656,0.0,0.107251,1.385693,-0.406533
281,XOM,95.120003,73.180000,0.253286,0.074361,Exxon Mobil Corp.,Energy,Energy,Integrated Oil & Gas,XOM,3.85,Energy,90.260002,23.44,-0.612049,0.0,-0.608024,-0.860790,-0.406533


In [361]:
volatility_pe_df.head()


,symbol,volatility,added_volatility,Name,Sector,GICS Sector,GICS Sub Industry,Ticker,EPS,Industry,Price,PE,std_pe
0,TDG,0.228652,0.142494,TransDigm Group,Industrials,Industrials,Aerospace & Defense,TDG,2.77,Industrials,248.960007,89.88,2.446119
1,ALK,0.758006,0.250980,Alaska Air Group Inc,Industrials,Industrials,Airlines,ALK,6.61,Industrials,88.730003,13.42,-0.485459
2,LRCX,0.666669,0.137728,Lam Research,Information Technology,Information Technology,Semiconductor Equipment,LRCX,5.75,Information Technology,105.730003,18.39,-0.373890
3,CPB,0.200839,0.069950,Campbell Soup,Consumer Staples,Consumer Staples,Packaged Foods & Meats,CPB,1.82,Consumer Staples,60.470001,33.23,0.231245
4,K,0.181806,0.086913,Kellogg Co.,Consumer Staples,Consumer Staples,Packaged Foods & Meats,K,1.74,Consumer Staples,73.709999,42.36,0.569531
